In [1]:
# ok now I want to seee how I got crushed by kvCache
from vllm.config.model import _get_and_verify_max_len
_get_and_verify_max_len??

Signature:
_get_and_verify_max_len(
    hf_config: Any,
    tokenizer_config: dict | None,
    max_model_len: int | None,
    disable_sliding_window: bool,
    sliding_window: int | None,
    spec_target_max_model_len: int | None = None,
    encoder_config: Optional[Any] = None,
) -> int
Source:   
def _get_and_verify_max_len(
    hf_config: PretrainedConfig,
    tokenizer_config: dict | None,
    max_model_len: int | None,
    disable_sliding_window: bool,
    sliding_window: int | None,
    spec_target_max_model_len: int | None = None,
    encoder_config: Any | None = None,
) -> int:
    """Get and verify the model's maximum length."""
    derived_max_model_len = float("inf")
    possible_keys = [
        # OPT
        "max_position_embeddings",
        # GPT-2
        "n_positions",
        # MPT
        "max_seq_len",
        # ChatGLM2
        "seq_length",
        # Command-R
        "model_max_length",
        # Whisper
        "max_target_positions",
        # Others
        "m

In [ ]:
# ok let's dig into step by step
# what is a max_model_len
# The usage I have seen is at KV cache allocation

# How it derives it startup

#1) ok one good thing
"""so what I understood it iterates through the config of is 
about to get loaded model and looks for they key which is 
associated with model internal max_seq_len which total prompt + generatedoutput"""
# possible_keys = [
#         # OPT
#         "max_position_embeddings",
#         # GPT-2
#         "n_positions",
#         # MPT
#         "max_seq_len",
#         # ChatGLM2
#         "seq_length",
#         # Command-R
#         "model_max_length",
#         # Whisper
#         "max_target_positions",
#         # Others
#         "max_sequence_length",
#         "max_seq_length",
#         "seq_len",
#     ]
#     # Choose the smallest "max_length" from the possible keys
#     max_len_key = None
#     for key in possible_keys:
#         max_len = getattr(hf_config, key, None)
#         if max_len is not None:
#             max_len_key = key if max_len < derived_max_model_len else max_len_key
#             derived_max_model_len = min(derived_max_model_len, max_len)
#     # For Command-R / Cohere, Cohere2 / Aya Vision models
#     if tmp_max_len := getattr(hf_config, "model_max_length", None):
#         max_len_key = "model_max_length"
#         derived_max_model_len = tmp_max_len

# 2) ok let's keep going so what is this if (
#         disable_sliding_window
#         and sliding_window is not None
#         and sliding_window < derived_max_model_len
#     ):
#         max_len_key = "sliding_window"
#         derived_max_model_len = sliding_window


# So if model has sliding_window = 4096 in its config:

# Default: vLLM uses sliding window attention (parameter is False)
# User passes --disable-sliding-window: vLLM uses full attention, but caps max_model_len to 4096

# The check only triggers when user explicitly asks to disable it.

# Full attention (position 10):
# Token 10 can see: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#                    ↑  all previous tokens
# Sliding window = 4 (position 10):
# Token 10 can see: [_, _, _, _, _, _, 6, 7, 8, 9]
#                                      ↑  only last 4
# So how does it "remember" earlier tokens?
# Layer by layer propagation:
# Layer 1: token 6 saw tokens [2,3,4,5]
# Layer 2: token 6 now "contains" info about 2,3,4,5
# Layer 3: token 10 sees token 6, indirectly gets info about 2,3,4,5
# Information about old tokens is baked into the hidden states of newer tokens. 
# It's indirect.



In [3]:
# in search of KV Cache calculation
from vllm.config import CacheConfig
CacheConfig??

Init signature:
CacheConfig(
    block_size: Annotated[Literal[1, 8, 16, 32, 64, 128, 256], SkipValidation()] = None,
    gpu_memory_utilization: float = 0.9,
    swap_space: float = 4,
    cache_dtype: Literal['auto', 'bfloat16', 'fp8', 'fp8_e4m3', 'fp8_e5m2', 'fp8_inc', 'fp8_ds_mla'] = 'auto',
    is_attention_free: bool = False,
    num_gpu_blocks_override: int | None = None,
    sliding_window: int | None = None,
    enable_prefix_caching: bool = True,
    prefix_caching_hash_algo: Literal['sha256', 'sha256_cbor'] = 'sha256',
    cpu_offload_gb: float = 0,
    calculate_kv_scales: bool = False,
    cpu_kvcache_space_bytes: int | None = None,
    mamba_page_size_padded: int | None = None,
    mamba_block_size: int | None = None,
    mamba_cache_dtype: Literal['auto', 'float32'] = 'auto',
    mamba_ssm_cache_dtype: Literal['auto', 'float32'] = 'auto',
    kv_sharing_fast_prefill: bool = False,
    kv_cache_memory_bytes: int | None = None,
    kv_offloading_size: float | None = None,


In [9]:
from vllm.v1.engine.core import EngineCore
EngineCore??

Init signature:
EngineCore(
    vllm_config: vllm.config.vllm.VllmConfig,
    executor_class: type[vllm.v1.executor.abstract.Executor],
    log_stats: bool,
    executor_fail_callback: collections.abc.Callable | None = None,
)
Source:        
class EngineCore:
    """Inner loop of vLLM's Engine."""

    def __init__(
        self,
        vllm_config: VllmConfig,
        executor_class: type[Executor],
        log_stats: bool,
        executor_fail_callback: Callable | None = None,
    ):
        # plugins need to be loaded at the engine/scheduler level too
        from vllm.plugins import load_general_plugins

        load_general_plugins()

        self.vllm_config = vllm_config
        if vllm_config.parallel_config.data_parallel_rank == 0:
            logger.info(
                "Initializing a V1 LLM engine (v%s) with config: %s",
                VLLM_VERSION,
                vllm_config,
            )

        self.log_stats = log_stats

        # Setup Model.
        self.model

In [11]:
from vllm.v1.core.kv_cache_utils import get_kv_cache_configs
get_kv_cache_configs??

Signature:
get_kv_cache_configs(
    vllm_config: vllm.config.vllm.VllmConfig,
    kv_cache_specs: list[dict[str, vllm.v1.kv_cache_interface.KVCacheSpec]],
    available_memory: list[int],
) -> list[vllm.v1.kv_cache_interface.KVCacheConfig]
Source:   
def get_kv_cache_configs(
    vllm_config: VllmConfig,
    kv_cache_specs: list[dict[str, KVCacheSpec]],
    available_memory: list[int],
) -> list[KVCacheConfig]:
    """
    Generates the KV cache configurations for a model.
    Since we use a shared centralized controller for all workers, we need the
    `kv_cache_config` to be consistent across all workers to make sure
    the KV cache allocation can be applied to all workers. However, different
    workers may have different memory available, and different type of layers
    (when pipeline parallel is enabled). To handle the difference between
    workers, the current implementation is:
    1. Merge the KV cache specs of all workers to get the KVCacheSpecs for
       the whole model.

In [12]:
from vllm.v1.core.kv_cache_utils import check_enough_kv_cache_memory
check_enough_kv_cache_memory??

Signature:
check_enough_kv_cache_memory(
    vllm_config: vllm.config.vllm.VllmConfig,
    kv_cache_spec: dict[str, vllm.v1.kv_cache_interface.KVCacheSpec],
    available_memory: int,
)
Source:   
def check_enough_kv_cache_memory(
    vllm_config: VllmConfig,
    kv_cache_spec: dict[str, KVCacheSpec],
    available_memory: int,
):
    """
    Checks whether `available_memory` is enough for the KV cache to hold at
    least one request with the model's max_model_len.

    Args:
        vllm_config: The global VllmConfig
        kv_cache_spec: The kv cache spec of each attention layer in the model
        available_memory: Memory available for KV cache in bytes.

    Raises:
        ValueError: If there is not enough memory available for the KV cache.
    """

    # No need to check for available memory if the kv_cache_spec is empty
    if not kv_cache_spec:
        return

    if available_memory <= 0:
        raise ValueError(
            "No available memory for the cache blocks. "
  

In [13]:
from vllm.v1.core.kv_cache_utils import max_memory_usage_bytes
max_memory_usage_bytes??

Signature:
max_memory_usage_bytes(
    vllm_config: vllm.config.vllm.VllmConfig,
    kv_cache_specs: collections.abc.Iterable[vllm.v1.kv_cache_interface.KVCacheSpec],
) -> int
Source:   
def max_memory_usage_bytes(
    vllm_config: VllmConfig, kv_cache_specs: Iterable[KVCacheSpec]
) -> int:
    """
    Get the maximum memory usage in bytes for the given KV cache specs.
    """
    return sum(spec.max_memory_usage_bytes(vllm_config) for spec in kv_cache_specs)
File:      /usr/local/lib/python3.10/dist-packages/vllm/v1/core/kv_cache_utils.py
Type:      function

In [14]:
from vllm.v1.kv_cache_interface import KVCacheSpec
KVCacheSpec??

Init signature: KVCacheSpec(block_size: int) -> None
Source:        
@dataclass(frozen=True)
class KVCacheSpec:
    """
    A base class for specifying the KV cache format of one layer.
    """

    # number of tokens in a block
    block_size: int

    @property
    def page_size_bytes(self) -> int:
        """
        The size of a page with `block_size` tokens in bytes.

        Returns:
            The page size
        """
        raise NotImplementedError

    def max_memory_usage_bytes(self, vllm_config: VllmConfig) -> int:
        """
        The maximum possible memory usage of this KV cache in bytes.

        Returns:
            The KV cache size in bytes
        """
        raise NotImplementedError

    def copy_with_new_block_size(self, block_size: int) -> Self:
        """
        Create a new KVCacheSpec from self but replacing the block size.
        """
        return replace(self, block_size=block_size)

    @classmethod
    def merge(cls, specs: list[Self]) -> Self:

In [15]:
from vllm.v1.kv_cache_interface import AttentionSpec
AttentionSpec??

Init signature:
AttentionSpec(
    block_size: int,
    num_kv_heads: int,
    head_size: int,
    dtype: torch.dtype,
) -> None
Docstring:      AttentionSpec(block_size: int, num_kv_heads: int, head_size: int, dtype: torch.dtype)
Source:        
@dataclass(frozen=True)
class AttentionSpec(KVCacheSpec):
    num_kv_heads: int
    head_size: int
    dtype: torch.dtype

    @property
    def page_size_bytes(self) -> int:
        return (
            2
            * self.block_size
            * self.num_kv_heads
            * self.head_size
            * get_dtype_size(self.dtype)
        )
File:           /usr/local/lib/python3.10/dist-packages/vllm/v1/kv_cache_interface.py
Type:           type
Subclasses:     FullAttentionSpec, ChunkedLocalAttentionSpec, SlidingWindowSpec, EncoderOnlyAttentionSpec, CrossAttentionSpec

In [17]:
from vllm.v1.kv_cache_interface import FullAttentionSpec
FullAttentionSpec??

Init signature:
FullAttentionSpec(
    block_size: int,
    num_kv_heads: int,
    head_size: int,
    dtype: torch.dtype,
    sliding_window: int | None = None,
    attention_chunk_size: int | None = None,
) -> None
Docstring:      FullAttentionSpec(block_size: int, num_kv_heads: int, head_size: int, dtype: torch.dtype, sliding_window: int | None = None, attention_chunk_size: int | None = None)
Source:        
@dataclass(frozen=True)
class FullAttentionSpec(AttentionSpec):
    sliding_window: int | None = None
    attention_chunk_size: int | None = None
    """
    When hybrid allocator is disabled and the model contains both full 
    attention layers and sliding window attention layers, sliding 
    window attention are regarded as full attention in KV cache manager 
    (blocks are allocated for all tokens), while computed as sliding window 
    attention in model runner.
    In this case, we use FullAttentionSpec and record the sliding window size.
    Default to None for not usin

In [ ]:
# oh my god now I circled back to
# KVCacheSpec object 
# I need to understand where its initialized


In [21]:
from vllm.v1.worker.gpu_model_runner import GPUModelRunner
GPUModelRunner.get_kv_cache_spec??

Signature: GPUModelRunner.get_kv_cache_spec(self) -> dict[str, vllm.v1.kv_cache_interface.KVCacheSpec]
Source:   
    def get_kv_cache_spec(self) -> dict[str, KVCacheSpec]:
        """
        Generates the KVCacheSpec by parsing the kv cache format from each
        Attention module in the static forward context.
        Returns:
            KVCacheSpec: A dictionary mapping layer names to their KV cache
            format. Layers that do not need KV cache are not included.
        """
        if has_ec_transfer() and get_ec_transfer().is_producer:
            return {}
        kv_cache_spec: dict[str, KVCacheSpec] = {}
        layer_type = cast(type[Any], AttentionLayerBase)
        attn_layers = get_layers_from_vllm_config(self.vllm_config, layer_type)
        for layer_name, attn_module in attn_layers.items():
            if isinstance(attn_module, Attention) and (
                kv_tgt_layer := attn_module.kv_sharing_target_layer_name
            ):
                # The layer do

In [29]:
from vllm.config.vllm import get_layers_from_vllm_config
get_layers_from_vllm_config??

Signature:
get_layers_from_vllm_config(
    vllm_config: vllm.config.vllm.VllmConfig,
    layer_type: type[~T],
    layer_names: list[str] | None = None,
) -> dict[str, ~T]
Source:   
def get_layers_from_vllm_config(
    vllm_config: VllmConfig,
    layer_type: type[T],
    layer_names: list[str] | None = None,
) -> dict[str, T]:
    """
    Get layers from the vLLM config.

    Args:
        vllm_config: The vLLM config.
        layer_type: The type of the layer to get.
        layer_names: The names of the layers to get. If None, return all layers.
    """

    if layer_names is None:
        layer_names = list(vllm_config.compilation_config.static_forward_context.keys())

    forward_context = vllm_config.compilation_config.static_forward_context

    return {
        layer_name: forward_context[layer_name]
        for layer_name in layer_names
        if isinstance(forward_context[layer_name], layer_type)
    }
File:      /usr/local/lib/python3.10/dist-packages/vllm/config/vllm.py


In [31]:
Attention.__init__??

Signature:
Attention.__init__(
    self,
    num_heads: int,
    head_size: int,
    scale: float,
    num_kv_heads: int | None = None,
    alibi_slopes: list[float] | None = None,
    cache_config: vllm.config.cache.CacheConfig | None = None,
    quant_config: vllm.model_executor.layers.quantization.base_config.QuantizationConfig | None = None,
    logits_soft_cap: float | None = None,
    per_layer_sliding_window: int | None = None,
    prefix: str = '',
    attn_type: str = 'decoder',
    kv_sharing_target_layer_name: str | None = None,
    attn_backend: type[vllm.attention.backends.abstract.AttentionBackend] | None = None,
    **extra_impl_args,
) -> None
Source:   
    def __init__(
        self,
        num_heads: int,
        head_size: int,
        scale: float,
        num_kv_heads: int | None = None,
        alibi_slopes: list[float] | None = None,
        cache_config: CacheConfig | None = None,
        quant_config: QuantizationConfig | None = None,
        logits_soft_cap:

In [ ]:
compilation_config.static_forward_context[prefix] = self

# Each Attention layer adds itself to the global dict during construction!!!



In [32]:
#Ok now full math

In [33]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct")
print(f"num_layers: {config.num_hidden_layers}")
print(f"num_attention_heads: {config.num_attention_heads}")
print(f"num_kv_heads: {getattr(config, 'num_key_value_heads', config.num_attention_heads)}")
print(f"hidden_size: {config.hidden_size}")
print(f"max_position_embeddings: {config.max_position_embeddings}")

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

num_layers: 32
num_attention_heads: 32
num_kv_heads: 32
hidden_size: 4096
max_position_embeddings: 16384


In [36]:

# head_size = hidden_size/num_attention_heads

head_size = 4096//32
head_size

128

In [39]:
# page_size_bytes = 2*block_size*num_kv_head*head_size*dtypes_bytes
# block_size = 16 vLLM and dtype bfloat16 = 2 bytes 
page_size_bytes = 2*16*32*128*2
page_size_bytes

262144

In [42]:
# num_blocks = ceildiv(max_position/block_size)
import math
num_blocks = math.ceil(16384/16)
num_blocks

1024

In [45]:
# memory_per_layer = num_blocks*page_size_bytes
memory_per_layer = 1024*262144
memory_per_layer

268435456

In [46]:
# across all layers in bytes
total = 32*268435456
total

8589934592

In [49]:
total/(1024*1024*1024)
# max needed memory
# in gb

8.0